In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math

from IPython.core.display import display,HTML
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



try:
    import cPickle as pickle  ##################################
   #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
import time  



import plotly.plotly as py


# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)
################



##### for getting geolocation data  and to calculate distance between two geolocations
import requests
import json
import geopy.distance   


In [ ]:
#time.sleep(36000)    # waiting time in seconds\n",

In [2]:
path = "/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file = 'Dropbox_datafile_complete_new_ranking.csv'
#input_file = 'Dropbox_datafile_may22_2017_modified_added_univ_country_geolocation_RUCC_population_density_when_available_career_stage_GINI_folder_act_categories.csv'
df = pd.read_csv(path+input_file, sep=';',na_values=["NAN","-1","null"],low_memory=False, parse_dates=['folder_creation_date','date_last_change']) # set header=0 if i wanna pass it my own list of header names
df.drop('Unnamed: 0', axis=1, inplace=True)



print df.shape

(1403349, 38)


In [ ]:
list(df[df.folder_id == 768713].Cleaned_university_name.unique())

In [ ]:

#df.shape
#df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1'],inplace=True,axis=1)
df.columns

In [ ]:
df.world_ranking.unique()

In [3]:
dict_var_dict_bins_categories={}
dict_var_dict_bins_categories["group_total_publ"]={}
dict_var_dict_bins_categories["group_num_papers_last"]={}
dict_var_dict_bins_categories["group_num_citations"]={}



dict_var_dict_bins_categories["group_total_publ"]["1.0-2.0"]=1
dict_var_dict_bins_categories["group_total_publ"]["2.0-8.0"]=2
dict_var_dict_bins_categories["group_total_publ"]["8.0-22.0"]=3
dict_var_dict_bins_categories["group_total_publ"]["22.0-56.0"]=4
dict_var_dict_bins_categories["group_total_publ"]["56.0-1350.0"]=5



dict_var_dict_bins_categories["group_num_papers_last"]["0.0-1.0"]=1
dict_var_dict_bins_categories["group_num_papers_last"]["1.0-5.0"]=2
dict_var_dict_bins_categories["group_num_papers_last"]["5.0-18.0"]=3
dict_var_dict_bins_categories["group_num_papers_last"]["18.0-1216.0"]=4



dict_var_dict_bins_categories["group_num_citations"]["0.0-1.0"]=1
dict_var_dict_bins_categories["group_num_citations"]["1.0-30.0"]=2
dict_var_dict_bins_categories["group_num_citations"]["30.0-229.0"]=3
dict_var_dict_bins_categories["group_num_citations"]["229.0-1183.0"]=4
dict_var_dict_bins_categories["group_num_citations"]["1183.0-146545.0"]=5



# group_total_publ
# 1.0-2.0        84560
# 2.0-8.0        42471
# 8.0-22.0       51958
# 22.0-56.0      66337
# 56.0-1350.0    70420

# group_num_papers_last
# 0.0-1.0        102479
# 1.0-5.0         51978
# 5.0-18.0        74631
# 18.0-1216.0     86658


# group_num_citations
# 0.0-1.0            72323
# 1.0-30.0           48710
# 30.0-229.0         57632
# 229.0-1183.0       64114
# 1183.0-146545.0    72967

In [4]:

def remove_nans(lista):
   # print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            if np.isnan(lista[i]) == True:  # (when nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
                pass#print "nan found"
            else: # (when not-nan):           
                aux_lista.append(lista[i])
            #print "no problem"
        except : ## whenever it is a STR            
            pass            
            
            
    #print aux_lista   
    return aux_lista


############################3



def remove_nans_for_strings(lista):
   # print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            if np.isnan(lista[i]) == True:  # (when nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
                pass#print "nan found"
            else: # (when not-nan):           
                aux_lista.append(lista[i])
            #print "no problem"
        except : ## whenever it is a STR            
             aux_lista.append(lista[i])#pass             
            
            
    #print aux_lista   
    return aux_lista




###############

def remove_nans_replace_by_zeros(lista):
   # print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            if np.isnan(lista[i]) == True:  # (when nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
                aux_lista.append(0.)        #pass#print "nan found"
            else: # (when not-nan):           
                aux_lista.append(float(lista[i]))
            #print "no problem"
        except : ## whenever it is a STR            
            pass             
            
            
    #print aux_lista   
    return aux_lista




###### get dict with  folder_id lifespan:
def remove_nans_replace_intervals_with_starting_value(lista):
    #print "\n",lista
    aux_lista=[]
    for i in range(len(lista)):    
       # print lista[i]
        try:
            if np.isnan(lista[i]) == True:  # (when/ nan):   # OJO!!! NO SIRVE SI HAGO: if lista[I]== np.nan   :(   !!!
                pass#print "nan found"
        except TypeError: # (when not-nan):
            #print lista[i]   
            try:  
                aux_lista.append(float(lista[i]))
                #print "no problem"
            except ValueError:  # sometimes, the ranking is 75-82
                #print "issue found"
                value=lista[i]
#                 v1=float(value.split("-")[0])
#                 v2=float(value.split("-")[1])
                
                new_value=float(value.split("-")[0])#np.median([v1,v2])  
                aux_lista.append(new_value)
            
    #print aux_lista   
    return aux_lista


######


In [5]:
# Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Unnamed: 0.1', u'folder_id',
#        u'num_folder_members', u'folder_creation_date', u'date_last_change',
#        u'user_id', u'email_domain', u'num_adds', u'num_edits', u'num_deletes',
#        u'major_content_type', u'major_content_ext', u'group_total_publ',
#        u'group_num_papers_last', u'group_num_citations', u'folder_lifespan',
#        u'simplified_domain', u'University_Name', u'Cleaned_university_name',
#        u'Country', u'Geolocation', u'lat', u'long', u'zip',
#        u'Rural_Urban_Continuum_Code', u'State_Name', u'Description',
#        u'Population_2010', u'Land-Sq-Mi', u'Density_Per_Sq_Mile',
#        u'world_ranking', u'national_ranking', u'career_stage',
#        u'median_num_publ', u'median_num_last_auth', u'folder_activity_GINI',
#        u'category_total_publ', u'category_total_last_auth',
#        u'category_total_cit'],
#       dtype='object')

################

#to get dict for the folder_id vs lifespan    'folder_lifespan'

def grouping(input_df,dict_folder_id_folder_attr):
    #print input_df
   
    folder_id=input_df.folder_id.iloc[0]
    dict_folder_id_folder_attr[folder_id]={}
    
    folder_lifespan=input_df.folder_lifespan.iloc[0]
    dict_folder_id_folder_attr[folder_id]["folder_lifespan"]=folder_lifespan
    
    
    num_folder_members=input_df.num_folder_members.iloc[0]
    dict_folder_id_folder_attr[folder_id]["num_folder_members"]=num_folder_members
    
    
    list_univ_rankings=remove_nans(list(input_df.world_ranking))
    dict_folder_id_folder_attr[folder_id]["folder_univ_median_ranking"]=np.median(list_univ_rankings)
    dict_folder_id_folder_attr[folder_id]["folder_univ_SD_ranking"]=np.std(list_univ_rankings)
    dict_folder_id_folder_attr[folder_id]["list_ranking"]=list(input_df.world_ranking)
    dict_folder_id_folder_attr[folder_id]["list_univ"]=list(input_df.Cleaned_university_name)
    dict_folder_id_folder_attr[folder_id]["list_countries"]=list(input_df.Country)
    
    
    
    dict_folder_id_folder_attr[folder_id]["multi_univ_collab"]=1
    lista_univ=list(input_df.Cleaned_university_name.unique())       
    dict_folder_id_folder_attr[folder_id]["multi_univ_collab"]=len(lista_univ)
    
    
    
    dict_folder_id_folder_attr[folder_id]["num_countries"]=1
    lista_countr=list(input_df.Country.unique())       
    dict_folder_id_folder_attr[folder_id]["num_countries"]=len(lista_countr)
    
    
    
    
    try:
        most_common=max(set(list_univ_rankings), key=list_univ_rankings.count)
    except ValueError:  # when empty list
        most_common=np.nan
    dict_folder_id_folder_attr[folder_id]["most_common_univ_ranking"]=most_common
    
            
    
    folder_activity_GINI= input_df.folder_activity_GINI.iloc[0]
    dict_folder_id_folder_attr[folder_id]["folder_activity_GINI"]=folder_activity_GINI
    
    
    tot_num_adds=sum(remove_nans_replace_by_zeros(list(input_df.num_adds)))
    dict_folder_id_folder_attr[folder_id]["folder_tot_num_adds"]=tot_num_adds
    
    
    tot_num_edits=sum(remove_nans_replace_by_zeros(list(input_df.num_edits)))
    dict_folder_id_folder_attr[folder_id]["folder_tot_num_edits"]=tot_num_edits
    
        
    tot_num_deletes=sum(remove_nans_replace_by_zeros(list(input_df.num_deletes)))
    dict_folder_id_folder_attr[folder_id]["folder_tot_num_deletes"]=tot_num_deletes
    
    
    tot_act=tot_num_adds+tot_num_edits+tot_num_deletes
    dict_folder_id_folder_attr[folder_id]["folder_tot_act"]=tot_act
    #dict_folder_id_folder_attr[folder_id]["list_act"]=list(input_df.num_adds)+list(input_df.num_edits)+list(input_df.num_deletes)
    
    
    
    
    
    
    lista=remove_nans_for_strings(list(input_df.career_stage))  
    try:
        most_common=max(set(lista), key=lista.count)
    except ValueError:  # when empty list
        most_common=np.nan
    dict_folder_id_folder_attr[folder_id]["most_common_career_stage"]=most_common
    
    try:    
        dict_folder_id_folder_attr[folder_id]["fraction_SR"]=lista.count("sr")/float(len(lista))
    except ZeroDivisionError:
        dict_folder_id_folder_attr[folder_id]["fraction_SR"]=np.nan
    
    #dict_folder_id_folder_attr[folder_id]["list_careers"]=list(input_df.career_stage)
    
    
    df_selection1= input_df[input_df['career_stage'] == "sr"]
    act_sr=sum(remove_nans_replace_by_zeros(list(df_selection1.num_deletes))) +\
    sum(remove_nans_replace_by_zeros(list(df_selection1.num_edits)))+ \
    sum(remove_nans_replace_by_zeros(list(df_selection1.num_adds)))
    try:
        dict_folder_id_folder_attr[folder_id]["fraction_work_by_SR"]=  act_sr/float(tot_act)
    except ZeroDivisionError:    
        dict_folder_id_folder_attr[folder_id]["fraction_work_by_SR"]= np.nan
        
        
    lista=remove_nans_for_strings(list(input_df.career_stage))
    if lista.count("sr")==0:
        dict_folder_id_folder_attr[folder_id]["fraction_work_by_SR"]= np.nan
    
    
    
                        
    lista=remove_nans(list(input_df.category_total_publ))
    try:
        most_common=max(set(lista), key=lista.count)
    except ValueError:  # empty list
        most_common=np.nan
    dict_folder_id_folder_attr[folder_id]["most_common_categ_num_publ"]=most_common
    
    #print "\n",input_df.category_total_last_auth
    lista=remove_nans(list(input_df.category_total_last_auth))
    #print "lista:",lista
    try:
        most_common=max(set(lista), key=lista.count)
    except ValueError:  # empty list
        most_common=np.nan
    dict_folder_id_folder_attr[folder_id]["most_common_categ_num_last_auth"]=most_common
#     print "most common", most_common
#     raw_input()
    
     
    lista=remove_nans(list(input_df.category_total_cit))
    try:
        most_common=max(set(lista), key=lista.count)
    except ValueError:  # empty list
        most_common=np.nan
    dict_folder_id_folder_attr[folder_id]["most_common_num_cit"]=most_common
    
   
    
    
 
 ##########################


dict_folder_id_folder_attr={}
for folder_id in df['folder_id'].unique():
    df_selection= df[df['folder_id'] == folder_id]
    grouping(df_selection,dict_folder_id_folder_attr)
    dict_folder_id_folder_attr[folder_id]["number_active_members"]=len(df_selection)
    #print folder_id


print "done"


/home/juliaponcela/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning:

Mean of empty slice.

/home/juliaponcela/anaconda2/lib/python2.7/site-packages/numpy/core/_methods.py:82: RuntimeWarning:

Degrees of freedom <= 0 for slice



done


In [7]:

dict_folder_id_folder_attr[768713]

{'folder_activity_GINI': nan,
 'folder_lifespan': 1584,
 'folder_tot_act': 0.0,
 'folder_tot_num_adds': 0.0,
 'folder_tot_num_deletes': 0.0,
 'folder_tot_num_edits': 0.0,
 'folder_univ_SD_ranking': 0.0,
 'folder_univ_median_ranking': 51.0,
 'fraction_SR': nan,
 'fraction_work_by_SR': nan,
 'list_ranking': [51.0, 51.0],
 'list_univ': ['university of southern california',
  'university of southern california'],
 'most_common_career_stage': nan,
 'most_common_categ_num_last_auth': nan,
 'most_common_categ_num_publ': nan,
 'most_common_num_cit': nan,
 'most_common_univ_ranking': 51.0,
 'multi_univ_collab': 1,
 'num_folder_members': 2,
 'number_active_members': 2}

In [ ]:
df.head()


#df[['folder_id','folder_creation_date','date_last_change','folder_lifespan','user_id','num_adds', 'num_edits', 'num_deletes', 'group_total_publ', 'group_num_papers_last','group_num_citations', 'folder_lifespan', 'national_ranking', 'career_stage', 'median_num_publ', 'median_num_last_auth','folder_activity_GINI','category_total_publ','category_total_last_auth', 'category_total_cit']].head(100)

In [ ]:
df.columns
print sorted( df.world_ranking.unique())

In [6]:
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr.pickle'
with open(pickle_name, 'wb') as handle:
    pickle.dump(dict_folder_id_folder_attr, handle)
print "written:", pickle_name



df_from_dict = pd.DataFrame.from_dict(dict_folder_id_folder_attr,orient='index')
df_from_dict.to_csv(pickle_name.strip("dict_").strip(".pickle")+".csv", sep=';')
print "written:", pickle_name.strip("dict_").strip(".pickle")+".csv"


written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr.pickle
written: /home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_folder_id_folder_attr.csv


In [ ]:
#df_from_dict.drop('list_careers', axis=1, inplace=True)

In [ ]:
df_from_dict.head(100)
#df_folder_aggr = pd.DataFrame.from_dict(dict_folder_id_folder_attr,orient='index')


In [ ]:
#dict_folder_id_folder_attr[592732]
df_from_dict.fraction_SR.unique()

In [ ]:
# Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Unnamed: 0.1', u'folder_id',
#        u'num_folder_members', u'folder_creation_date', u'date_last_change',
#        u'user_id', u'email_domain', u'num_adds', u'num_edits', u'num_deletes',
#        u'major_content_type', u'major_content_ext', u'group_total_publ',
#        u'group_num_papers_last', u'group_num_citations', u'folder_lifespandict_user_id_user_attr[user_id]["user_univ_ranking"]=univ_ra',
#        u'simplified_domain', u'University_Name', u'Cleaned_university_name',
#        u'Country', u'Geolocation', u'lat', u'long', u'zip',
#        u'Rural_Urban_Continuum_Code', u'State_Name', u'Description',
#        u'Population_2010', u'Land-Sq-Mi', u'Density_Per_Sq_Mile',
#        u'world_ranking', u'national_ranking', u'career_stage',
#        u'median_num_publ', u'median_num_last_auth', u'folder_activity_GINI',
#        u'category_total_publ', u'category_total_last_auth',
#        u'category_total_cit'],
#       dtype='object')


          
            
          

################

#to get dict for the user_id vs attr

def grouping(input_df,dict_user_id_user_attr):
    #print input_df
   
    user_id=input_df.user_id.iloc[0]
    dict_user_id_user_attr[user_id]={}
    
    
    dict_user_id_user_attr[user_id]["mean_folder_lifespan"]=np.mean(list(input_df.folder_lifespan))
    
  
    
    try:
        univ_ranking=float(input_df.world_ranking.iloc[0])
    except ValueError:  # if 101-150
        univ_ranking=float(input_df.world_ranking.iloc[0].split("-")[0])
    dict_user_id_user_attr[user_id]["user_univ_ranking"]=univ_ranking
    
    
#     folder_activity_GINI= input_df.folder_activity_GINI.iloc[0]
#     dict_user_id_user_attr[user_id]["folder_activity_GINI"]=folder_activity_GINI
    
    
    tot_num_adds=sum(remove_nans_replace_by_zeros(list(input_df.num_adds)))
    dict_user_id_user_attr[user_id]["user_tot_num_adds"]=tot_num_adds
    
    
    tot_num_edits=sum(remove_nans_replace_by_zeros(list(input_df.num_edits)))
    dict_user_id_user_attr[user_id]["user_tot_num_edits"]=tot_num_edits
    
    
    
    
    
    ##########  FALTA CREAR: NUMERO TOTAL DE COLABORADORES (UNIQUE USER_IDS, NOT NUM_FOLDER_MEMBERS)
    
    
        
    tot_num_deletes=sum(remove_nans_replace_by_zeros(list(input_df.num_deletes)))
    dict_user_id_user_attr[user_id]["user_tot_num_deletes"]=tot_num_deletes
    
    
    tot_act=tot_num_adds+tot_num_edits+tot_num_deletes
    dict_user_id_user_attr[user_id]["user_tot_act"]=tot_act
    
    
    career_stage=input_df.career_stage.iloc[0]   
    dict_user_id_user_attr[user_id]["career_stage"]=career_stage
                
    
    category_total_publ=input_df.category_total_publ.iloc[0]       
    dict_user_id_user_attr[user_id]["category_total_publ"]=category_total_publ
    
    
    category_total_last_auth=input_df.category_total_last_auth.iloc[0]       
    dict_user_id_user_attr[user_id]["category_total_last_auth"]=category_total_last_auth
    
   
    category_total_cit=input_df.category_total_cit.iloc[0]      
    dict_user_id_user_attr[user_id]["category_total_cit"]=category_total_cit
    
     
    
    
 
 #########################


dict_user_id_user_attr={}
for user_id in df['user_id'].unique():
    df_selection= df[df['user_id'] == user_id]
    grouping(df_selection,dict_user_id_user_attr)
    #print folder_id
    dict_user_id_user_attr[user_id]["number_folders"]=len(df_selection)

print "done"



In [ ]:
#df.head(100).sort_values(by="user_id", axis=0, ascending=True)



In [ ]:
pickle_name='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/dict_user_id_user_attr.pickle'
with open(pickle_name, 'wb') as handle:
    pickle.dump(dict_user_id_user_attr, handle)
print "written:", pickle_name

df_from_dict = pd.DataFrame.from_dict(dict_user_id_user_attr,orient='index')
df_from_dict.to_csv(pickle_name.strip("dict_").strip(".pickle")+".csv", sep=';')
print "written:", pickle_name.strip("dict_").strip(".pickle")+".csv"


In [ ]:
# for user in dict_user_id_user_attr:
#     if dict_user_id_user_attr[user]['category_total_cit']>2:
#         print dict_user_id_user_attr[user]
#         raw_input()
